In [103]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/spam.csv


In [104]:
DATA_FILE = '/kaggle/input/spam.csv'
df = pd.read_csv(DATA_FILE,encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [105]:
tags  = df.v1
texts = df.v2

In [106]:
encoder = LabelBinarizer()
encoder.fit(df['v1'])
y = encoder.transform(df['v1'])

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec       = TfidfVectorizer(max_features=100)
tfidf_mat = vec.fit_transform(texts).toarray()
#print(type(tfidf_mat),tfidf_mat.shape)  # 5572 doc, tfidf 100 dimension

In [110]:
model = Sequential()
model.add(Dense(64,input_shape=(100,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                6464      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_7 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_8 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [111]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

In [112]:
model.fit(tfidf_mat,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 1s 126us/step - loss: 0.3846 - acc: 0.8555 - val_loss: 0.2217 - val_acc: 0.8726
Epoch 2/10
4457/4457 [==============================] - 0s 71us/step - loss: 0.1734 - acc: 0.9327 - val_loss: 0.1144 - val_acc: 0.9650
Epoch 3/10
4457/4457 [==============================] - 0s 73us/step - loss: 0.1172 - acc: 0.9621 - val_loss: 0.1036 - val_acc: 0.9650
Epoch 4/10
4457/4457 [==============================] - 0s 71us/step - loss: 0.1060 - acc: 0.9666 - val_loss: 0.0970 - val_acc: 0.9686
Epoch 5/10
4457/4457 [==============================] - 0s 69us/step - loss: 0.0945 - acc: 0.9724 - val_loss: 0.0951 - val_acc: 0.9686
Epoch 6/10
4457/4457 [==============================] - 0s 69us/step - loss: 0.0887 - acc: 0.9731 - val_loss: 0.0951 - val_acc: 0.9695
Epoch 7/10
4457/4457 [==============================] - 0s 73us/step - loss: 0.0819 - acc: 0.9737 - val_loss: 0.0933 - val_acc: 0.9704
Epoch 

In [147]:
i = 5567
tst = [df['v2'][i]]
tst_ = vec.transform(tst).toarray()
print(df['v2'][i])
print("-------------------")
if model.predict(tst_)[0][0] > 0.50:
    print("SPAM")
else:
    print("HAM")

df.tail()

This is the 2nd time we have tried 2 contact u. U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.
-------------------
SPAM


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN
5571,ham,Rofl. Its true to its name,NaN,NaN,NaN
